# Aplicando CNN para classificação de fissura em concreto armado e pavimento

Nestes notebooks será aplicado redes pré-treinadas para detecção de fissura, neste dataset de imagens, o foco é comparar a redes pré-treinadas e ver qual seria a rede com o melhor desempenho

In [1]:
#importando as blibiotecas

from tensorflow import keras
from keras.layers import Dense, Flatten
from tensorflow.keras.models import Model, Sequential
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.applications import VGG16, ResNet50,DenseNet121

import tensorflow as tf

In [2]:
# importando a blibioteca split-folders para dividir as imagens em treino e teste
!pip install split-folders

import splitfolders
splitfolders.ratio('/kaggle/input/concrete-and-pavement-crack-images', 
                   output="output", seed=1337, ratio=(.7, 0.3))

/opt/conda/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()


Copying files: 30000 files [00:40, 749.60 files/s] 


In [3]:
BATCH = 26
IMG_WIDTH = 96
IMG_HEIGHT = 96

In [4]:
train_loader = tf.keras.preprocessing.image_dataset_from_directory(
"./output/train",
seed=123,
image_size=(IMG_HEIGHT, IMG_WIDTH),
batch_size=BATCH,label_mode= 'categorical'
)

test_loader = tf.keras.preprocessing.image_dataset_from_directory(
"./output/val",
seed=123,
image_size=(IMG_HEIGHT, IMG_WIDTH),
batch_size=BATCH, label_mode= 'categorical')

Found 21000 files belonging to 2 classes.
Found 9000 files belonging to 2 classes.


## Criando o modelo DenseNet121

In [5]:
model = DenseNet121(input_shape=(96, 96, 3), weights='imagenet', include_top=False,
             pooling='max',classifier_activation='softmax')
model.layers[0].trainable = False
model = Model(inputs=model.input, outputs=model.get_layer(index=-2).output)

29084464/29084464 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step


In [6]:
#Colocando as duas últimas camadas
newTop = Sequential()
newTop.add(Flatten(input_shape=model.output_shape[1:]))
newTop.add(Dense(2, activation='softmax'))

/opt/conda/lib/python3.10/site-packages/keras/src/layers/reshaping/flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [7]:
metricas = ['accuracy',
            keras.metrics.Precision(name='Pre'),keras.metrics.Recall(name='Rec')] 
#métricas que vão ser utilizadas para teste

In [8]:
model = Model(inputs=model.input, outputs=newTop(model.output))
model.summary()

optimizer = 'adam'
model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=metricas)

Model: "functional_5"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 96, 96, 3) │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ zero_padding2d      │ (None, 102, 102,  │          0 │ input_layer[0][0] │
│ (ZeroPadding2D)     │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_conv (Conv2D) │ (None, 48, 48,    │      9,408 │ zero_padding2d[0… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_bn            │ (None, 48, 48,    │        256 │ conv1_conv[0][0]  │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_relu          │ (None, 48, 48,    │          0 │ conv1_bn[0][0]    │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ zero_padding2d_1    │ (None, 50, 50,    │          0 │ conv1_relu[0][0]  │
│ (ZeroPadding2D)     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ pool1               │ (None, 24, 24,    │          0 │ zero_padding2d_1… │
│ (MaxPooling2D)      │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_0_bn   │ (None, 24, 24,    │        256 │ pool1[0][0]       │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_0_relu │ (None, 24, 24,    │          0 │ conv2_block1_0_b… │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_1_conv │ (None, 24, 24,    │      8,192 │ conv2_block1_0_r… │
│ (Conv2D)            │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_1_bn   │ (None, 24, 24,    │        512 │ conv2_block1_1_c… │
│ (BatchNormalizatio… │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_1_relu │ (None, 24, 24,    │          0 │ conv2_block1_1_b… │
│ (Activation)        │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_2_conv │ (None, 24, 24,    │     36,864 │ conv2_block1_1_r… │
│ (Conv2D)            │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_concat │ (None, 24, 24,    │          0 │ pool1[0][0],      │
│ (Concatenate)       │ 96)               │            │ conv2_block1_2_c… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block2_0_bn   │ (None, 24, 24,    │        384 │ conv2_block1_con… │
│ (BatchNormalizatio… │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block2_0_relu │ (None, 24, 24,    │          0 │ conv2_block2_0_b… │
│ (Activation)        │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block2_1_conv │ (None, 24, 24,    │     12,288 │ conv2_block2_0_r

 Total params: 7,055,938 (26.92 MB)

 Trainable params: 6,972,290 (26.60 MB)

 Non-trainable params: 83,648 (326.75 KB)

In [9]:
AUTOTUNE = tf.data.AUTOTUNE

train_dataset = train_loader.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)
test_dataset = test_loader.cache().prefetch(buffer_size=AUTOTUNE)

In [10]:
callback = keras.callbacks.EarlyStopping(monitor='loss',
                                              patience=2)
history = model.fit(train_dataset,epochs=5,batch_size=100,callbacks=[callback])

Epoch 1/5


I0000 00:00:1733235409.220520    1267 service.cc:145] XLA service 0x7d7748002c40 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1733235409.220587    1267 service.cc:153]   StreamExecutor device (0): Tesla P100-PCIE-16GB, Compute Capability 6.0
I0000 00:00:1733235468.553349    1267 asm_compiler.cc:369] ptxas warning : Registers are spilled to local memory in function 'input_reduce_select_slice_fusion', 8 bytes spill stores, 8 bytes spill loads

I0000 00:00:1733235468.687127    1267 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


808/808 ━━━━━━━━━━━━━━━━━━━━ 224s 129ms/step - Pre: 0.9596 - Rec: 0.9596 - accuracy: 0.9596 - loss: 0.1496
Epoch 2/5
808/808 ━━━━━━━━━━━━━━━━━━━━ 48s 59ms/step - Pre: 0.9775 - Rec: 0.9775 - accuracy: 0.9775 - loss: 0.0673
Epoch 3/5
808/808 ━━━━━━━━━━━━━━━━━━━━ 46s 57ms/step - Pre: 0.9938 - Rec: 0.9938 - accuracy: 0.9938 - loss: 0.0224
Epoch 4/5
808/808 ━━━━━━━━━━━━━━━━━━━━ 48s 59ms/step - Pre: 0.9953 - Rec: 0.9953 - accuracy: 0.9953 - loss: 0.0156
Epoch 5/5
808/808 ━━━━━━━━━━━━━━━━━━━━ 49s 61ms/step - Pre: 0.9942 - Rec: 0.9942 - accuracy: 0.9942 - loss: 0.0179


In [11]:
model.evaluate(test_dataset)

347/347 ━━━━━━━━━━━━━━━━━━━━ 21s 38ms/step - Pre: 0.9923 - Rec: 0.9923 - accuracy: 0.9923 - loss: 0.0257


[0.023441432043910027,
 0.9931111335754395,
 0.9931111335754395,
 0.9931111335754395]